## CANTILEVER BEAM ANALYSIS WITH AXIAL LOADING
### Displacement calculation using FEniCS

This notebook performs structural analysis of a cantilever beam subjected to axial loading using the FEniCS finite element library. The analysis calculates the displacement field and saves the results in XDMF format for visualization.

In [1]:

# 1. IMPORT LIBRARIES AND MODULES
# =============================================================================
from dolfin import *
import numpy as np
import os

In [2]:
# 2. IMPORT THE MESH
# =============================================================================
# Create output directory if it doesn't exist
output_dir = "post-processing/output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load mesh from existing file
mesh_file_path = "mesh/mesh.xdmf"
mesh = Mesh()
with XDMFFile(mesh_file_path) as infile:
    infile.read(mesh)

In [3]:
# 3. DEFINE MATERIAL PROPERTIES
# =============================================================================
# Material parameters (Steel)
E = 200e9           # Young's modulus [Pa]
nu = 0.3            # Poisson's ratio
mu = E / (2 * (1 + nu))      # Shear modulus
lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))  # Lame parameter

In [4]:
# 4. DEFINE FUNCTION SPACES
# =============================================================================
# Vector function space for displacement (P1 elements)
V = VectorFunctionSpace(mesh, 'CG', 1)

In [5]:
# Define the location of the loading and boundary condition
mesh_coords = mesh.coordinates()
x_min, x_max = mesh_coords[:, 0].min(), mesh_coords[:, 0].max()

In [6]:
# 5. DEFINE BOUNDARY CONDITIONS AND LOADING
# =============================================================================
# Define boundary conditions using actual mesh dimensions
def clamped_boundary(x, on_boundary):
    """Fixed boundary at x = x_min"""
    return on_boundary and near(x[0], x_min, 1e-6)

# Apply fixed boundary condition (zero displacement)
bc = DirichletBC(V, Constant((0, 0, 0)), clamped_boundary)

# Define loading
axial_load = 1e6  # Applied axial load [N] - reduced from 1e12 to reasonable value
boundary_parts = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundary_parts.set_all(0)

# Mark the loaded boundary using consistent coordinates
class LoadedBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], x_max, 1e-6)

loaded_boundary_marker = LoadedBoundary()
loaded_boundary_marker.mark(boundary_parts, 1)

# Define measure for boundary integration
ds = Measure('ds', domain=mesh, subdomain_data=boundary_parts)

In [7]:
# 6. DEFINE THE MAIN PHYSICS - VARIATIONAL FORMULATION
# =============================================================================
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

# Define strain and stress tensors
def epsilon(u):
    """Strain tensor"""
    return 0.5 * (nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    """Stress tensor"""
    return lmbda * tr(epsilon(u)) * Identity(3) + 2 * mu * epsilon(u)

# Define variational problem
a = inner(sigma(u), epsilon(v)) * dx
L = dot(Constant((axial_load, 0, 0)), v) * ds(1)

# 7. SOLVE THE PROBLEM
# =============================================================================
# Compute solution
u_solution = Function(V)
solve(a == L, u_solution, bc)

In [8]:
# 8. SAVE RESULTS - DISPLACEMENT FIELD ONLY
# =============================================================================
# Save displacement field using with statement
u_solution.rename("Displacement", "u")
output_file_path = os.path.join(output_dir, 'displacement.xdmf')
with XDMFFile(output_file_path) as displacement_file:
    displacement_file.parameters["flush_output"] = True
    displacement_file.parameters["functions_share_mesh"] = True
    displacement_file.write(u_solution, 0.0)

print(f'The analysis is complete and the result is saved in the file: {output_file_path}')

The analysis is complete and the result is saved in the file: post-processing/output/displacement.xdmf


In [9]:
# 9. CALCULATE AND PRINT MAXIMUM DISPLACEMENT
# =============================================================================
# Get the maximum displacement magnitude from the solution
u_magnitude = sqrt(dot(u_solution, u_solution))
# Project to scalar function space to get a Function object
V_scalar = FunctionSpace(mesh, 'CG', 1)
u_magnitude_func = project(u_magnitude, V_scalar)
max_displacement = u_magnitude_func.vector().max()

print(f'Maximum displacement: {max_displacement:.6e} m')

Maximum displacement: 4.968995e-05 m
